# Import de librairies

In [2]:
import keras.optimizers
%matplotlib notebook
import pandas as pd
import numpy as np
import tensorflow  as tf
from sklearn.model_selection import train_test_split

In [3]:
x_train=pd.read_csv('xtrain.csv')
x_test=pd.read_csv('xtest.csv')
y_train=pd.read_csv('ytrain_NpxebDC.csv')

/Users/juanfelipearias/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Segmentation of intersected data between x_train and y_train. This will be based on the assumption that the state of the stations wont change within the hour, therefore we change the  frequency from 15m to 1 h

In [4]:
y_train=y_train.loc[y_train.timestamp.isin(x_train.timestamp)]
x_train=x_train.loc[x_train.timestamp.isin(y_train.timestamp)] 

## Treating mising values

In [5]:
for i in x_train.columns:
    if  x_train[i].isna().any():
        print("There are "+str(len(x_train[i].loc[x_train[i].isna()]))+" missing values in the column:" + i+"proportion: "+str((len(x_train[i].loc[x_train[i].isna()])/len(x_train[i]))*100)[:5]+"%")

There are 3 missing values in the column:temperatureproportion: 0.038%
There are 6 missing values in the column:wind_speedproportion: 0.076%
There are 6 missing values in the column:wind_gust_speedproportion: 0.076%
There are 48 missing values in the column:rain_levelproportion: 0.615%
There are 58 missing values in the column:traffic_stateproportion: 0.743%
There are 58 missing values in the column:average_flowproportion: 0.743%


In [6]:
x_train.hist(['rain_level','average_flow','temperature','wind_speed','wind_gust_speed'])

<IPython.core.display.Javascript object>

array([[<AxesSubplot:title={'center':'rain_level'}>,
        <AxesSubplot:title={'center':'average_flow'}>],
       [<AxesSubplot:title={'center':'temperature'}>,
        <AxesSubplot:title={'center':'wind_speed'}>],
       [<AxesSubplot:title={'center':'wind_gust_speed'}>, <AxesSubplot:>]],
      dtype=object)

## Prepration of exogene variables

In [7]:
x_train.temperature.fillna(method='ffill',inplace=True)
x_train.rain_level.fillna(method='ffill',inplace=True)
x_train.average_flow.fillna(method='ffill',inplace=True)
x_train.wind_speed.fillna(method='ffill',inplace=True)
x_train.wind_gust_speed.fillna(method='ffill',inplace=True)
x_train.traffic_state.fillna(method='ffill',inplace=True)
reps={'Available':4,'Charging':3,'Passive':2,'Offline':1,'Down':0}
x_test.temperature.fillna(method='ffill',inplace=True)
x_test.rain_level.fillna(method='ffill',inplace=True)
x_test.average_flow.fillna(method='ffill',inplace=True)
x_test.wind_speed.fillna(method='ffill',inplace=True)
x_test.wind_gust_speed.fillna(method='ffill',inplace=True)
x_test.traffic_state.fillna(method='ffill',inplace=True)

# clean={i:reps for i in y_train.columns[1:]}
# y_train.replace(clean,inplace=True)

x_train.replace({'traffic_state':{'Fluid':0,"Busy":1,"Heavy":2}},inplace=True)
x_test.replace({'traffic_state':{'Fluid':0,"Busy":1,"Heavy":2}},inplace=True)

## Preparation of the the target values

In [8]:
y_doom=pd.get_dummies(y_train.drop(['timestamp'],axis='columns'))
states=['Available','Charging','Passive','Offline','Down']
cols=list(y_train.drop(['timestamp'],axis='columns').columns)
doom_cols=[i+'_'+j for i in cols for j in states]
for i in doom_cols:
    if i not in  list(y_doom.columns):
        y_doom[i]=0
y_doom=y_doom[doom_cols]
y_doom['timestamp']=pd.to_datetime(y_train.timestamp)
y_doom.set_index('timestamp',inplace=True)

In [9]:
idx=pd.date_range(list(y_doom.index)[0],list(y_doom.index)[-1],freq="H")[~pd.date_range(list(y_doom.index)[0],list(y_doom.index)[-1],freq="H").isin(y_doom.index)]
pd.DataFrame({i:np.nan for i in y_doom.columns},index=idx)
y_doom=y_doom.append(pd.DataFrame({i:np.nan for i in y_doom.columns},index=idx)).sort_index()
#  Forward fill for the missing values
y_doom.fillna(method='ffill',inplace=True)

In [10]:
def prepare_ts_date(df,look_back=1):
    dataX, dataY = [], []
    for i in range(len(df)-look_back-1):
        a = df.iloc[i:(i+look_back)].to_numpy()
        dataX.append(a)
        dataY.append(df.iloc[i + look_back])
    return np.array(dataX), np.array(dataY)

In [11]:
x,y=prepare_ts_date(y_doom,1)


In [12]:
x.shape

(8398, 1, 1365)

In [25]:
# x_train.drop(['timestamp'],axis='columns',inplace=True)
# X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_doom)
model= tf.keras.models.Sequential([
    tf.keras.layers.LSTM(300, input_shape=(x.shape[1],x.shape[2]),return_sequences=True),
    tf.keras.layers.LSTM(300,return_sequences=True),
    tf.keras.layers.Dense(1365,activation='softmax')

])

In [26]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 300)            1999200   
                                                                 
 lstm_7 (LSTM)               (None, 1, 300)            721200    
                                                                 
 dense_5 (Dense)             (None, 1, 1365)           410865    
                                                                 
Total params: 3,131,265
Trainable params: 3,131,265
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["acc"])


In [28]:
history=model.fit(x,y.reshape(y.shape[0],1,y.shape[1]),epochs=200)

Epoch 1/200
263/263 [==============================] - 12s 34ms/step - loss: 1863.3699 - acc: 0.0000e+00
Epoch 2/200
263/263 [==============================] - 7s 27ms/step - loss: 2243.1328 - acc: 0.0000e+00
Epoch 3/200
263/263 [==============================] - 6s 23ms/step - loss: 2667.6387 - acc: 0.0000e+00
Epoch 4/200
263/263 [==============================] - 6s 24ms/step - loss: 3111.1052 - acc: 0.0000e+00
Epoch 5/200
263/263 [==============================] - 7s 25ms/step - loss: 3562.3196 - acc: 0.0000e+00
Epoch 6/200
263/263 [==============================] - 6s 24ms/step - loss: 4010.3467 - acc: 0.0000e+00
Epoch 7/200
263/263 [==============================] - 5s 21ms/step - loss: 4477.3262 - acc: 0.0000e+00
Epoch 8/200
263/263 [==============================] - 6s 21ms/step - loss: 4940.4751 - acc: 0.0000e+00
Epoch 9/200
263/263 [==============================] - 7s 25ms/step - loss: 5397.9121 - acc: 0.0000e+00
Epoch 10/200
263/263 [==============================] - 6s 25ms

KeyboardInterrupt: 

In [85]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,Y_train)
y_hat=clf.predict(X_val)

In [86]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_val,y_hat)

0.0

In [87]:
y_hat2=clf.predict(x_test.drop(['timestamp'],axis='columns'))

In [135]:
tf.config.run_functions_eagerly(True)

In [92]:
x_train

temperature  wind_speed  wind_gust_speed  rain_level  traffic_state  \
0             8.7        11.0             24.0         0.0              0   
1             8.8         9.0             20.0         0.0              0   
2             8.9         8.0             18.0         0.0              0   
3             8.8         7.0             18.0         0.0              0   
4             8.8        10.0             23.0         0.0              0   
...           ...         ...              ...         ...            ...   
8395         14.1         7.0             10.0         0.0              0   
8396         14.2         7.0             13.0         0.0              0   
8397         13.4         5.0             11.0         0.0              0   
8398         12.5         4.0             10.0         0.0              0   
8399         12.0         5.0              6.0         0.0              0   

      average_flow  
0            218.1  
1            144.3  
2            110.0  
3            143.8  
4            315.1  
...            ...  
8395         370.3  
8396         286.7  
8397         185.5  
8398         118.9  
8399          84.6  

[7804 rows x 6 columns]